In [2]:
import torch
from datasets import load_dataset
from torch.cuda.tunable import enable
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:

model_name = "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [14]:
len(model.model.layers)

28

In [10]:

# dataset = load_dataset("opencompass/AIME2025", "AIME2025-I", split="test")
dataset = load_dataset("allenai/ai2_arc", "ARC-Challenge", split="test")
model.to("cuda")

correct = 0
total = 0

# it = iter(dataset)
# next(it)
# item = next(it)
dataset = dataset.take(100)

for item in dataset:
    question = item["question"]
    # reference_answer = item["answer"]
    choices = item["choices"]
    answer_key = item["answerKey"]

    for i, choice in enumerate(choices['text']):
        label = choices['label'][i]
        question += f"{label}. {choice}\n"

    messages= [
        {
            'content': 'You are a helpful assistant. Answer with a SINGLE letter without any additional formatting.',
            'role': 'system',
        },
        {
            'content': question,
            'role': 'user',
        },
    ]


    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    # Generate response
    outputs = model.generate(**inputs, max_new_tokens=1, do_sample=False)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    predicted_answer = generated_text[-1]

    if answer_key.lower() == predicted_answer.lower():
        correct += 1
    else:
        print(f"Correct answer: {answer_key}, predicted answer: {predicted_answer}")
    total += 1

    print(f"Processed {total}/{len(dataset)}...")

accuracy = (correct / total) * 100 if total > 0 else 0
print(f"\nAccuracy: {accuracy:.2f}% ({correct}/{total})")


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Processed 1/100...
Processed 2/100...
Correct answer: C, predicted answer: B
Processed 3/100...
Processed 4/100...
Correct answer: D, predicted answer: C
Processed 5/100...
Correct answer: B, predicted answer: D
Processed 6/100...
Processed 7/100...
Processed 8/100...
Processed 9/100...
Processed 10/100...
Processed 11/100...
Processed 12/100...
Correct answer: C, predicted answer: B
Processed 13/100...
Correct answer: C, predicted answer: B
Processed 14/100...
Correct answer: C, predicted answer: D
Processed 15/100...
Processed 16/100...
Processed 17/100...
Correct answer: B, predicted answer: A
Processed 18/100...
Processed 19/100...
Correct answer: C, predicted answer: A
Processed 20/100...
Processed 21/100...
Processed 22/100...
Processed 23/100...
Correct answer: C, predicted answer: A
Processed 24/100...
Processed 25/100...
Correct answer: D, predicted answer: A
Processed 26/100...
Processed 27/100...
Processed 28/100...
Correct answer: C, predicted answer: B
Processed 29/100...
